In [1]:
# Import 
import os
import time
import json
import random
import datetime
import numpy as np
import pandas as pd

from IPython.display import display

from pathlib import Path
from tyssue import config
from tyssue import Sheet
from tyssue.io import hdf5
from tyssue.io.hdf5 import load_datasets
from tyssue.topology import all_rearangements
from tyssue.core.history import HistoryHdf5
from tyssue.dynamics import SheetModel as basemodel
from tyssue.solvers.quasistatic import QSSolver
from tyssue.draw import sheet_view
from tyssue.draw.plt_draw import quick_edge_draw
from tyssue.draw.ipv_draw import sheet_view as ipv_draw 

from tyssue.behaviors.event_manager import EventManager
from tyssue.behaviors.sheet.apoptosis_events import apoptosis
from tyssue.behaviors.sheet.basic_events import reconnect

import matplotlib.pyplot as plt
%matplotlib inline

try:
    import seaborn as sns
except ImportError:
    print('Seaborn not found')



In [2]:
from polarity.dynamics import model
from polarity.apoptosis import apoptosis
from polarity.dynamics import EllipsoidLameGeometry as geom
from polarity.toolbox import (init,
                              define_fold_position,
                              apoptosis_ventral,
                              apoptosis_lateral,
                              apoptosis_dorsal,
                             )


In [3]:
SIM_DIR = Path('/home/admin-suz/Documents/Simulations/test_history')

sim_save_dir = SIM_DIR

try:
    os.mkdir(sim_save_dir)
except FileExistsError:
    pass

In [4]:
solver = QSSolver(with_t1=False, with_t3=False, with_collisions=False)

In [5]:
from tyssue.dynamics import units, effectors, model_factory
from polarity.dynamics import RadialTension
model = model_factory(
    [
        RadialTension,
        effectors.LineTension,
        #effectors.FaceContractility,
        effectors.FaceAreaElasticity,
        effectors.LumenVolumeElasticity,
    ], effectors.FaceAreaElasticity)

In [6]:
# Create tissue tissue 

def load_before_apopto(anchor_tension=0):

    #h5store = '../examples/superegg_final.hf5'
    h5store = '../examples/tissue_prepolarized.hf5'

    datasets = load_datasets(h5store,
                             data_names=['face', 'vert', 'edge'])
    
    with open('../examples/superegg_final.json', 'r+') as fp:
        specs = json.load(fp)
    
    sheet = Sheet('spherical', datasets, specs)
    sheet.settings['geometry'] = "spherical"
    sheet.settings['lumen_vol_elasticity'] = 3.e-6
    #sheet.settings['lumen_prefered_vol'] = sheet.settings['lumen_prefered_vol'] - sheet.settings['lumen_prefered_vol']*10/100

    return sheet

sheet = load_before_apopto(0)

geom.update_all(sheet)



In [7]:
sheet.edge_df.line_tension *= 40.
sheet.edge_df.line_tension.unique()

array([40., 48., 24.])

In [8]:
res = solver.find_energy_min(sheet, geom, model, options={"gtol": 1e-8})

"""fig, ax = quick_edge_draw(sheet, ['z', 'x'],
                              alpha=0.7)
fig.set_size_inches(18.5, 10.5, forward=True)"""

"fig, ax = quick_edge_draw(sheet, ['z', 'x'],\n                              alpha=0.7)\nfig.set_size_inches(18.5, 10.5, forward=True)"

In [9]:
"""import ipyvolume as ipv
ipv.clear()
# code couleur profondeur du tissu
fig, mesh = sheet_view(sheet, mode="3D", coords=['z', 'x', 'y'],
                   edge={'color':sheet.vert_df.y*(sheet.vert_df.y>0),
                         'colormap':'viridis'})
fig"""
import ipyvolume as ipv
ipv.clear()
# code couleur profondeur du tissu
fig, mesh = sheet_view(sheet, mode="3D", coords=['z', 'x', 'y'],
                   edge={'color':'white'},
                      face={'color':'black', 'visible':True})
fig



Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0…

In [10]:
define_fold_position(sheet, fold_number = 1, position=[-8, 8])
print(len(sheet.face_df[sheet.face_df.fold==1]))

ipv.clear()
# code couleur profondeur du tissu
fig, mesh = sheet_view(sheet, mode="3D", coords=['z', 'x', 'y'],
                   edge={'color':'lightslategrey',
                         'visible':True},
                   face={'color': sheet.face_df.fold,
                        'colormap':'Reds',
                        'visible':True})
fig


112


Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0…

In [11]:
sheet.face_df['apoptosis'] = 0
apoptosis_ventral(sheet, 1)
apoptosis_lateral(sheet, 1)
apoptosis_dorsal(sheet, 1)

ipv.clear()
# code couleur profondeur du tissu
fig, mesh = sheet_view(sheet, mode="3D", coords=['z', 'x', 'y'],
                   edge={'color':'lightslategrey',
                         'visible':True},
                   face={'color': sheet.face_df.apoptosis,
                        'colormap':'Reds',
                        'visible':True})
fig

print(len(sheet.face_df[sheet.face_df.fold>0]))
print(len(sheet.face_df[sheet.face_df.apoptosis>0]))
print(len(sheet.face_df[sheet.face_df.apoptosis==1]))
print(len(sheet.face_df[sheet.face_df.apoptosis==2]))
print(len(sheet.face_df[sheet.face_df.apoptosis==3]))

112
35
21
11
3


In [12]:
def run_sim(sim_save_dir, sheet, constriction, radial_tension, iteration=0):
    
    dirname = '{}_constriction_{}_radialtension{}'.format(
        constriction,radial_tension, iteration)
    dirname = os.path.join(sim_save_dir, dirname)
    
    print('starting {}'.format(dirname))
    try:
        os.mkdir(dirname)
    except IOError:
        pass

    
    apoptosis_settings = {
        "shrink_rate": 1.1,
        "critical_area": 5.,
        "contract_rate": constriction,
        "basal_contract_rate": 1.01,
        "contract_span": 4,
        "radial_tension": radial_tension,
        "max_traction":30.,
        "current_traction":0.,
        "geom": geom,
    }
    
    
    # Add some information to the sheet and copy initial sheet
    sheet2 = sheet.copy(deep_copy=True)
    sheet2.face_df['id'] = sheet2.face_df.index.values
    sheet2.settings['apoptosis'] = apoptosis_settings
    
    #""" Initiale find minimal energy
    # To be sure we are at the equilibrium
    res = solver.find_energy_min(sheet2, geom, model, options={"gtol": 1e-8})


    sheet2 = apoptosis_process(dirname, solver, sheet2, geom, model)
    
    print('{} done'.format(dirname))
    print('~~~~~~~~~~~~~~~~~~~~~\n')

In [13]:
def apoptosis_process(dirname, solver, sheet, geom, model):
    
    # Initiate manager
    manager = EventManager('face')
    
    # Initiate history
    history = HistoryHdf5(sheet, 
                          extra_cols={"face": sheet.face_df.columns,
                                      "edge": list(sheet.edge_df.columns),
                                      "vert": list(sheet.vert_df.columns)},
                         hf5file=os.path.join(dirname, 'out.hf5'))
    
    #save settings
    pd.Series(sheet.settings).to_csv(os.path.join(dirname, 'settings.csv'))

    t = 0.
    stop = 150.
    
    # Add events to limit rosette 
    rosette_kwargs={
        'threshold_length':0.1,
        'p_4':0.2,
        'p_5p':0.2}
    manager.append(reconnect, )

        
    # Run simulation
    while t < stop:
        
        # Définir sur combien de pas de temps on veut que les cellules entre en apoptose...
        # Répartir un "pointeur" qui part du ventral et va en dorsal...
        # Déclenche les cellules apoptotiques au fur et à mesure...
        
        ## Partie à tester....
        time_of_last_apoptosis = 30.   
            
        phi_min = -t*max(np.abs(sheet.face_df.phi))/30+max(np.abs(sheet.face_df.phi))

        l_index_apoptosis_cell = sheet.face_df[(np.abs(sheet.face_df.phi)>phi_min) 
                                               & (sheet.face_df.apoptosis>0)
                                              ].index.values
        apopto_kwargs = sheet.settings['apoptosis'].copy()
        for c in l_index_apoptosis_cell:
            apopto_kwargs.update(
            {
                'face_id': c,
            }
            )
            manager.append(apoptosis, **apopto_kwargs)  

        
        if t%10 == 0:
            sheet.settings['lumen_prefered_vol'] *= 0.99        
        
        # Reset radial tension at each time step
        sheet.vert_df.radial_tension = 0.
        
        
        manager.execute(sheet)
        res = solver.find_energy_min(sheet, geom, model, options={"gtol": 1e-8})
        #all_rearangements(sheet)
        #res = solver.find_energy_min(sheet, geom, model, options={"gtol": 1e-8})

        # add noise on vertex position to avoid local minimal.
        sheet.vert_df[
            ['x', 'y']] += np.random.normal(scale=1e-3, size=(sheet.Nv, 2))
        geom.update_all(sheet)
        
    
        
        # Save result in each time step.
        figname = os.path.join(
            dirname, 'invagination_{:04.0f}.png'.format(t))
        hdfname = figname[:-3] + 'hf5'        
        hdf5.save_datasets(hdfname, sheet)

        history.record()

        manager.update()
        t += 1

    return sheet


In [25]:
a = dict(one=1, two=2, four =2,)
b = {'one': 1, 'two': 2, 'three': 3}
a == b


False

In [14]:
# Sequential Execution

from datetime import datetime

global_start=datetime.now()
print ("start : " + str(global_start))

radial_tension = [20]
#for rd in radial_tension:
contractility_percent = [8, 16]
contractility_rate = [1+c/100 for c in contractility_percent]

for c in contractility_rate:
    for rd in radial_tension:
        for iteration in range (0, 1):
            run_sim(sim_save_dir, sheet, c, rd, iteration)
        
        
        
global_end = datetime.now()
print ("end : " + str(global_end))
print ('Duree totale d execution : \n\t\t')
print (global_end-global_start)

start : 2019-08-29 14:18:22.933217
starting /home/admin-suz/Documents/Simulations/test_history/1.08_constriction_20_radialtension0


KeyboardInterrupt: 

In [15]:
history = HistoryHdf5.from_archive(os.path.join(sim_save_dir, '1.08_constriction_20_radialtension0/out.hf5'))

In [18]:
sheet_ = history.retrieve(1.)

In [20]:
sheet_.face_df

,x,y,z,num_sides,area,perimeter,is_alive,face_o,opposite,height,...,vol,phi,theta,area_elasticity,prefered_area,contractility,id,fold,apoptosis,time
face,,,,,,,,,,,,,,,,,,,,,
0,-2.871946,51.778906,-16.669112,7,38.676020,23.498657,1.0,0.0,-1.0,50.605056,...,1957.070965,1.572379,-0.091744,1.0,36.681015,1.52,0,0,0,1.0
1,-8.933800,50.984297,-14.052314,5,25.436732,20.630172,1.0,1.0,-1.0,49.744556,...,1261.678025,1.842156,-0.114298,1.0,28.737944,1.52,1,0,0,1.0
2,-11.393636,49.760554,-29.173083,6,51.561294,27.678723,1.0,2.0,-1.0,54.961387,...,2835.428295,1.572519,-0.224442,1.0,50.440289,1.52,2,0,0,1.0
3,18.205955,48.119467,0.338061,6,49.346472,27.158831,1.0,3.0,-1.0,47.656968,...,2351.405029,1.298173,-0.035314,1.0,48.099945,1.52,3,1,0,1.0
4,-6.328026,50.631790,1.149701,6,34.975095,22.739065,1.0,4.0,-1.0,47.184342,...,1650.417985,1.545096,0.032793,1.0,34.586170,1.52,4,1,2,1.0
5,-13.122444,48.990117,1.036949,5,52.052851,28.296689,1.0,5.0,-1.0,46.958911,...,2443.220886,1.822596,0.032789,1.0,52.420959,1.52,5,1,0,1.0
6,-8.657626,50.050315,-35.140009,6,47.488364,26.570619,1.0,6.0,-1.0,57.904254,...,2774.461691,1.842403,-0.246030,1.0,46.387180,1.52,6,0,0,1.0
7,-46.410477,19.323220,-6.849359,7,44.597662,25.162184,1.0,7.0,-1.0,46.901359,...,2090.273465,2.936277,0.032904,1.0,41.518856,1.52,7,1,0,1.0
8,-47.257750,16.728563,-17.589740,5,39.537814,24.523524,1.0,8.0,-1.0,49.281881,...,1953.529223,2.936277,-0.103347,1.0,41.361492,1.52,8,0,0,1.0
